## Finding $\alpha$

Bayesian purists look away!
We need to find a value of $\alpha$ that gets us to the sweet spot between minimising the misfit term and minimising the regularisation term - we want to be close to the minimum of the misfit whilst allowing the regularisation to still have an effect.
We therfore want to find a $\alpha$ in $J$ and $J'$ such that we sit at a turning point of a plot of 

$$\sqrt{J_{\text{model-data misfit}}^{\text{point}}} = \sqrt{\int_{\Omega} ( u_{\text{interpolated}} - u )^2 dx}$$

and

$$\sqrt{J_{\text{model-data misfit}}^{\text{field}}} = \sqrt{\int_{\Omega_v} ( u_{\text{obs}} - \mathcal{I}_{\text{P0DG}(\Omega_v)}(u) )^2 dx}$$

against

$$\frac{1}{\alpha}\sqrt{J_{\text{regularisation}}} = \sqrt{\int_\Omega|\nabla q|^2 dx}$$.

In [ ]:
from scipy.interpolate import (
    LinearNDInterpolator,
    NearestNDInterpolator,
    CloughTocher2DInterpolator,
    Rbf,
)

import matplotlib.pyplot as plt
import firedrake
import firedrake_adjoint

from firedrake import Constant, cos, sin

import numpy as np
from numpy import pi as π
from numpy import random

import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable

## Fake $q_{true}$

In [ ]:
mesh = firedrake.UnitSquareMesh(32, 32)

# Solution Space
V = firedrake.FunctionSpace(mesh, family='CG', degree=2)

# q (Control) Space
Q = firedrake.FunctionSpace(mesh, family='CG', degree=2)

seed = 1729
generator = random.default_rng(seed)

degree = 5
x = firedrake.SpatialCoordinate(mesh)

q_true = firedrake.Function(Q)
for k in range(degree):
    for l in range(int(np.sqrt(degree**2 - k**2))):
        Z = np.sqrt(1 + k**2 + l**2)
        ϕ = 2 * π * (k * x[0] + l * x[1])

        A_kl = generator.standard_normal() / Z
        B_kl = generator.standard_normal() / Z

        expr = Constant(A_kl) * cos(ϕ) + Constant(B_kl) * sin(ϕ)
        mode = firedrake.interpolate(expr, Q)

        q_true += mode

print('Made fake q_true')

## Fake $u_{true}$

In [ ]:
from firedrake import exp, inner, grad, dx
u_true = firedrake.Function(V)
v = firedrake.TestFunction(V)
f = Constant(1.0)
k0 = Constant(0.5)
bc = firedrake.DirichletBC(V, 0, 'on_boundary')
F = (k0 * exp(q_true) * inner(grad(u_true), grad(v)) - f * v) * dx
firedrake.solve(F == 0, u_true, bc)

print('Made fake u_true')

# Clear tape since don't need to have taped above
tape = firedrake_adjoint.get_working_tape()
tape.clear_tape()

## Generating Observational Data $u_{obs}$
We will investigate with $2^8 = 256$ measurements.

In [ ]:
i = 8
np.random.seed(0)

# Decide σ
signal_to_noise = 20
U = u_true.dat.data_ro[:]
u_range = U.max() - U.min()
σ = firedrake.Constant(u_range / signal_to_noise)

# Make random point cloud
num_points = 2**i
xs = np.random.random_sample((num_points,2))

# Generate "observed" data
print(f'Generating {num_points} fake observed values')
ζ = generator.standard_normal(len(xs))
u_obs_vals = np.array(u_true.at(xs)) + float(σ) * ζ

# Loop over $\hat{\sigma}$ values for each method

In [ ]:
# Setup methods and σ̂s

methods = ['nearest', 'linear', 'clough-tocher', 'gaussian']

σ̂_values = np.asarray([1.0,
    10.0,
    100.0,
    1000.0,
    10000.0,
    100000.0,
    0.1,
    20.0,
    30.0,
    40.0,
    22.0,
    24.0,
    26.0,
    28.0,
    25.0,
    27.0,
    23.0,
    23.5,
    27.5,
    22.5,
    22.75,
    27.75,
    22.25
    2.0,
    4.0,
    6.0,
    8.0,
    12.0,
    14.0,
    16.0,
    18.0,
    0.01,
    0.5,
    9.0,
    11.0,
    15.0])

Js = np.zeros((len(σ̂_values), len(methods)))
J_misfits = Js.copy()
J_regularisations = Js.copy()
J_misfit_times_vars = Js.copy()

# Loop over methods first to avoid recreating interpolators then σ̂s

for method_i, method in enumerate(methods):

    print(f'using {method} method')

    # Interpolating the mesh coordinates field (which is a vector function space)
    # into the vector function space equivalent of our solution space gets us
    # global DOF values (stored in the dat) which are the coordinates of the global
    # DOFs of our solution space. This is the necessary coordinates field X.
    print('Getting coordinates field X')
    Vc = firedrake.VectorFunctionSpace(mesh, V.ufl_element())
    X = firedrake.interpolate(mesh.coordinates, Vc).dat.data_ro[:]

    # Pick the appropriate "interpolate" method needed to create
    # u_interpolated given the chosen method
    print(f'Creating {method} interpolator')
    if method == 'nearest':
        interpolator = NearestNDInterpolator(xs, u_obs_vals)
    elif method == 'linear':
        interpolator = LinearNDInterpolator(xs, u_obs_vals, fill_value=0.0)
    elif method == 'clough-tocher':
        interpolator = CloughTocher2DInterpolator(xs, u_obs_vals, fill_value=0.0)
    elif method == 'gaussian':
        interpolator = Rbf(xs[:, 0], xs[:, 1], u_obs_vals, function='gaussian')
    print('Interpolating to create u_interpolated')
    u_interpolated = firedrake.Function(V, name=f'u_interpolated_{method}_{num_points}')
    u_interpolated.dat.data[:] = interpolator(X[:, 0], X[:, 1])

    for σ̂_i, σ̂_value in enumerate(σ̂_values):

        # Run the forward problem with q = 0 as first guess
        print('Running forward model')
        u = firedrake.Function(V)
        q = firedrake.Function(Q)
        bc = firedrake.DirichletBC(V, 0, 'on_boundary')
        F = (k0 * exp(q) * inner(grad(u), grad(v)) - f * v) * dx
        firedrake.solve(F == 0, u, bc)

        print(f'σ̂_i = {σ̂_i} σ̂_value = {σ̂_value}')

        σ̂ = firedrake.Constant(σ̂_value)

        # Two terms in the functional - note σ̂ in misfit term!
        misfit_expr = 0.5 * ((u_interpolated - u) / σ̂)**2
        α = firedrake.Constant(0.5)
        regularisation_expr = 0.5 * α**2 * inner(grad(q), grad(q))

        print('Assembling J\'\'')
        J = firedrake.assemble(misfit_expr * dx) + firedrake.assemble(regularisation_expr * dx)

        # Create reduced functional
        print('Creating q̂ and Ĵ\'\'')
        q̂ = firedrake_adjoint.Control(q)
        Ĵ = firedrake_adjoint.ReducedFunctional(J, q̂)

        # Minimise reduced functional
        print('Minimising Ĵ to get q_min')
        q_min = firedrake_adjoint.minimize(
            Ĵ, method='Newton-CG', options={'disp': True}
        )
        q_min.rename(name=f'q_min_{method}_{num_points}_{σ̂_value:.2}')

        # Get size of misfit term by solving PDE again using q_min
        print('Running forward model with q_min')
        u = firedrake.Function(V)
        bc = firedrake.DirichletBC(V, 0, 'on_boundary')
        F = (k0 * exp(q_min) * inner(grad(u), grad(v)) - f * v) * dx
        firedrake.solve(F == 0, u, bc)

        print("Reformulating J\'\' expressions")
        misfit_expr = 0.5 * ((u_interpolated - u) / σ̂)**2
        misfit_expr_times_var = 0.5 * (u_interpolated - u)**2
        regularisation_expr = 0.5 * α**2 * inner(grad(q_min), grad(q_min))
        
        print("Calculating J_misfit")
        J_misfit = firedrake.assemble(misfit_expr * dx)
        print(f'J_misfit = {J_misfit}')
        
        # Need to reform regularisation term with q_min instead of q
        print("Calculating J_regularisation")
        J_regularisation = firedrake.assemble(regularisation_expr * dx)
        print(f'J_regularisation = {J_regularisation}')

        print("Calculating J\'\'")
        J = J_misfit + J_regularisation
        print(f'J = {J}')

        print('Calculating J_misfit_times_var')
        J_misfit_times_var = firedrake.assemble(misfit_expr_times_var * dx)
        print(f'J_misfit_times_var = {J_misfit_times_var}')

        print(f'saving values: σ̂_i = {σ̂_i} method_i = {method_i}')
        J_misfits[σ̂_i, method_i] = J_misfit
        J_regularisations[σ̂_i, method_i] = J_regularisation
        Js[σ̂_i, method_i] = J
        J_misfit_times_vars[σ̂_i, method_i] = J_misfit_times_var
        
        print(f'Writing to q_min to q_mins checkpoint: σ̂_i = {σ̂_i} method_i = {method_i}')
        with firedrake.DumbCheckpoint("q_mins", mode=firedrake.FILE_UPDATE) as chk:
            chk.store(q_min)

        # Clear tape to avoid memory leak
        print('Clearing tape')
        tape.clear_tape()

# Save to CSV
Appending if we already have data

In [ ]:
import os
import csv

write_header = not os.path.isfile('Js.csv')
file = open('Js.csv', 'a')
writer = csv.writer(file)
if write_header:
    writer.writerow(['sigma_hat'] + methods)
writer.writerows(np.concatenate((σ̂_values[:, np.newaxis], Js), axis=1))
file.close()

write_header = not os.path.isfile('J_misfits.csv')
file = open('J_misfits.csv', 'a')
writer = csv.writer(file)
if write_header:
    writer.writerow(['sigma_hat'] + methods)
writer.writerows(np.concatenate((σ̂_values[:, np.newaxis], J_misfits), axis=1))
file.close()
    
write_header = not os.path.isfile('J_regularisations.csv')
file = open('J_regularisations.csv', 'a')
writer = csv.writer(file)
if write_header:
    writer.writerow(['sigma_hat'] + methods)
writer.writerows(np.concatenate((σ̂_values[:, np.newaxis], J_regularisations), axis=1))
file.close()

write_header = not os.path.isfile('J_misfit_times_vars.csv')
file = open('J_misfit_times_vars.csv', 'a')
writer = csv.writer(file)
if write_header:
    writer.writerow(['sigma_hat'] + methods)
writer.writerows(np.concatenate((σ̂_values[:, np.newaxis], J_misfit_times_vars), axis=1))
file.close()